# Assignment 4: Negamax with Alpha-Beta Pruning and Iterative Deepening

Still being developed, but you may get started on this when you are finished with Assignment 3.

# Table of Contents
* [Assignment 4: Negamax with Alpha-Beta Pruning and Iterative Deepening](#Assignment-4:-Negamax-with-Alpha-Beta-Pruning-and-Iterative-Deepening)
	* [Initial Code](#Initial-Code)
	* [Add moves counter](#Add-moves-counter)
	* [negamaxIDS](#negamaxIDS)
	* [negamaxIDSab](#negamaxIDSab)
	* [Grading](#Grading)
	* [Extra Credit](#Extra-Credit)


For this assignment, you will investigate the advantages of alpha-beta
pruning applied to Tic-Tac-Toe.  To do so, follow these steps.

## Initial Code

Check that the following function `playGame` runs
correctly. Notice that we are using *negamax* to find the best move for
Player X, but Player O, the opponent, is using function *opponent*
that follows the silly strategy of playing in the first open position.

In [4]:
%run MySolution.py
game = TTT()
playGame(game,opponent,10)
print("Moves Explored: " , game.getMovesExplored())

 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score inf
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 2 for score inf
X|O|X
-----
 | | 
-----
 | | 
Player O to 3
X|O|X
-----
O| | 
-----
 | | 
Player X to 4 for score inf
X|O|X
-----
O|X| 
-----
 | | 
Player O to 5
X|O|X
-----
O|X|O
-----
 | | 
Player X to 6 for score 1
X|O|X
-----
O|X|O
-----
X| | 
Moves Explored:  186


## Add moves counter

Evaluate the efficiency of the search by keepting track of the number of nodes explored, which is the same as the number of moves explored, during a game. Do this by adding a counter named `movesExplored` to the `TTT` constructor where it is initialized to 0 and increment the counter in the `TTT.makeMove` method.  Add a method `ttt.getNumberMovesExplored()` to get its current value.  So

    print('Number of moves explored', game.getMovesExplored())
    
will print the number of moves explored. You will not use a global variable for counting this time.

## negamaxIDS 

Write a new function named `negamaxIDS` that performs an iterative deepening negamax search.  Replace the first line in the `while` loop of `playGame` with

        score,move = negamaxIDS(game,depthLimit)
        
where `depthLimit` is now the maximum depth and multiple `negamax` searches are performed for depth limits of $1, 2, \ldots,$ maximum depth.  Make sure it works.

## negamaxIDSab

Now for the hardest part.  Make a new function `negamaxIDSab` by duplicating `negamaxIDS` and add the code to implement alpha-beta pruning.

Now duplicate the game playing loop so three complete tic-tac-toe games are played.  Call this new version `playGames`. For the first game, use `negamax`. For the second game, use `negamaxIDS`.  For the third game, use `negamaxIDSab`.  At the end of each game, print the number of X's in the final board, the number moves explored, the depth of the game which is the number of moves made by X and O, and the effective branching factor.  When you run `playGames` you should see the tic-tac-toe positions after each move and, after all games are done, a line for each game like

    negamax made 4 moves. 33216 moves explored for ebf(33216, 8) of 3.52    
    negamaxIDS made 4 moves. 186 moves explored for ebf(186, 8) of 1.73
    negamaxIDSab made 4 moves. 52 moves explored for ebf(52, 8) of 1.41
  
(Your results may be different.)

Here are some example results.

In [ ]:
# from A4mysolution import *

In [ ]:
def opponent(board):
    blanks = [i for i, mark in enumerate(board) if mark == ' ']
    # return blanks[-1]
    # return random.choice(blanks)
    return blanks[0]

In [ ]:
playGames(opponent, 25)

## Grading

Grading code is coming.  Stay Tuned

## Extra Credit 

Implement another game and repeat the above steps.

Implement a random move chooser as the opponent (Player O) and
determine how many times Player X can win against this opponent as an
average over multiple games.